In [11]:
import pandas as pd
import glob
from pathlib import Path
import numpy as np
from numpy.random import default_rng
rng = default_rng()


folder = "../../results_all/app_history_influxdb/"
filePrefix = "../../results_all/app_bootstrap_influx/app_bootstrap_influxdb"
print(folder)

start = 0.05
end = 0.2

numberOfSamples = 1000
CIsmall = 1

../../results_all/app_history_influxdb/


In [12]:
def resample(perfRuntimes1: pd.DataFrame,
            perfRuntimes2: pd.DataFrame,
            instanceRuns: np.ndarray,
            samples: int,
            numberOfSamples: int) -> np.ndarray:
    """
        Resamples performances using hierarchical bootstrapping for building confindence intervals

        Builds a tensor of random indices of a form numberOfSamples * instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Then uses these indices to choose from a performance runtimes tensor with a form instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Finally, reshapes resulting tensor to a matrix of form numberOfSamples * ( instanceRunsNumber * suiteRunsNumber * numberOfIterations)
        and calculates performance differences between medians.

        Parameters
        ----------
        perfRuntimes1 : performance runtimes of the first version.
        perfRuntimes2 : performance runtimes of the second version.
        instanceRuns : array of instanceRun numbers
        samples : number of drawn alues

        Returns
        -------
        np.ndarray
            array of performance differences with a shape numberOfSample * 1.

        """
    instanceRunsNumber = instanceRuns.shape[0]

    allRuntimes1 = np.ndarray((instanceRunsNumber, samples))
    allRuntimes2 = np.ndarray((instanceRunsNumber, samples))


    for instanceRun in instanceRuns:
        allRuntimes1[instanceRun - 1]= perfRuntimes1.loc[(perfRuntimes1['run'] == instanceRun),
                                                'latency (ms)'].to_numpy()
        allRuntimes2[instanceRun - 1]= perfRuntimes2.loc[(perfRuntimes2['run'] == instanceRun),
                                                'latency (ms)'].to_numpy()

    medians = []
    for i in range(numberOfSamples):
        #if (i % 1000 == 0):
        #    print(f"i is {i}")
        #Generate Random Arrays
        currentInstanceRun = rng.choice(instanceRuns, size=(instanceRunsNumber)) - 1
        currentRuntimes1 = rng.integers(samples, size=(samples, instanceRunsNumber))
        currentRuntimes2 = rng.integers(samples, size=(samples, instanceRunsNumber))
        #Bulk selection
        tmp1 = allRuntimes1[currentInstanceRun, currentRuntimes1]
        tmp1 = tmp1.reshape((instanceRunsNumber * samples))

        tmp2 = allRuntimes2[currentInstanceRun, currentRuntimes2]
        tmp2 = tmp2.reshape((instanceRunsNumber * samples))

        # Get median for both lists
        med1 = np.median(tmp1, axis=0)
        med2 = np.median(tmp2, axis=0)
        medians.append(med2/med1)

    return medians

def bootstrap(perfRuntimes1: pd.DataFrame,
            perfRuntimes2: pd.DataFrame):

    instanceRuns = perfRuntimes1.run.unique()
    numberOfInstanceRuns = len(instanceRuns)
    print(f"    Found {numberOfInstanceRuns} runs")
    R = resample(perfRuntimes1=perfRuntimes1,
                 perfRuntimes2=perfRuntimes2,
                 instanceRuns=instanceRuns,
                 samples=int(len(perfRuntimes1) / numberOfInstanceRuns),
                 numberOfSamples=numberOfSamples)

    print(f"    Bootstrapping done ({len(R)} elements in R).")

    # Find conf. intervals
    R.sort()

    small = int((numberOfSamples * CIsmall) / 100 / 2)
    if small == 0:
        small  = 1

    minSmall = R[small-1]
    minSmall = (minSmall - 1) * 100
    maxSmall = R[numberOfSamples-small-1]
    maxSmall = (maxSmall - 1) * 100
    return minSmall, maxSmall


all_runs = glob.glob(folder + "*")

for fullRunFolder in all_runs:
    runFolder = fullRunFolder + "/"
    all_numbers = glob.glob(runFolder + "*")
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(fullNumberFolder + "/*")
        size = len(all_files)

        if size != 17:
            print(f"{size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split("\\")

            #Parse run and commit number from path structure
            run = tmp[len(tmp)-2]
            run=run[3:]
            commitNumber = tmp[len(tmp)-1]
            print(f"Check Run {run} Number {commitNumber}")

            filename = Path(filePrefix + "_" + commitNumber + ".csv")
            skip = False

            if filename.is_file():
                print("  Already aggregated, skip")
            else:
                print("  Aggregating...")
                rows = []
                for i in range(3):
                    currFolder = folder + "run" + str((i+1)) + "/" + str(commitNumber)
                    tmp = glob.glob(currFolder + "/*")
                    noOfFiles = len(tmp)
                    print(f"    Current folder: {currFolder} with {noOfFiles} files")




                    if noOfFiles != 17:
                        print(f"ERROR: {noOfFiles} elements in folder {currFolder}, skip.")
                        skip = True
                    else:

                        #Read latency file for both types (old and new)
                        insertsOld = pd.read_csv(currFolder + "/latenciesInsertsOld.csv",
                                         header=None,
                                         names=["worker","batch","latency (ms)"])
                        insertsOld.sort_values(by=["batch"], inplace=True)
                        number = len(insertsOld.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        insertsOld = insertsOld.iloc[removeFirst:-removeLast , :]

                        for index, row in insertsOld.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        insertsNew = pd.read_csv(currFolder + "/latenciesInsertsNew.csv",
                                         header=None,
                                         names=["worker","batch","latency (ms)"])
                        insertsNew.sort_values(by=["batch"], inplace=True)
                        number = len(insertsNew.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        insertsNew = insertsNew.iloc[removeFirst:-removeLast , :]

                        for index, row in insertsNew.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries1Old = pd.read_csv(currFolder + "/latenciesQueries1Old.csv",
                                         header=None,
                                         names=["query","latency (ms)"])
                        queries1Old.sort_values(by=["query"], inplace=True)
                        number = len(queries1Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries1Old = queries1Old.iloc[removeFirst:-removeLast , :]


                        for index, row in queries1Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "simple queries",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries2Old = pd.read_csv(currFolder + "/latenciesQueries2Old.csv",
                                         header=None,
                                         names=["query","latency (ms)"])
                        queries2Old.sort_values(by=["query"], inplace=True)

                        number = len(queries2Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries2Old = queries2Old.iloc[removeFirst:-removeLast , :]

                        for index, row in queries2Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "group-by queries",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries1New = pd.read_csv(currFolder + "/latenciesQueries1New.csv",
                                         header=None,
                                         names=["query","latency (ms)"])
                        queries1New.sort_values(by=["query"], inplace=True)

                        number = len(queries1New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries1New = queries1New.iloc[removeFirst:-removeLast , :]

                        for index, row in queries1New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "simple queries",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries2New = pd.read_csv(currFolder + "/latenciesQueries2New.csv",
                                         header=None,
                                         names=["query","latency (ms)"])
                        queries2New.sort_values(by=["query"], inplace=True)

                        number = len(queries2New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries2New = queries2New.iloc[removeFirst:-removeLast , :]

                        for index, row in queries2New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "group-by queries",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })
                if not skip:

                    #Run bootstrapping for each type
                    result = []
                    df_latencies = pd.DataFrame(rows)
                    for type in df_latencies.type.unique():
                        print(f"  Running bootstrapping for type {type}...")

                        benchmarkMeasurements = df_latencies.loc[(df_latencies['type'].str.startswith(type, na=False))]
                        instanceRuns = benchmarkMeasurements.run.unique()
                        numberOfInstanceRuns = len(instanceRuns)
                        perfRuntimes1 = benchmarkMeasurements.loc[(benchmarkMeasurements['version'] == "base")]
                        perfRuntimes2 = benchmarkMeasurements.loc[(benchmarkMeasurements['version'] == "variation")]
                        print(f"    Found {len(perfRuntimes1)} and {len(perfRuntimes2)} values")
                        min, max = bootstrap(perfRuntimes1, perfRuntimes2)
                        print(f"    Min: {min} Max: {max} (Width: {max-min})")
                        result.append({
                            "commit":commitNumber,
                            "type": type,
                            "min" : min,
                            "max" : max
                        })

                    # Convert to data frame
                    df_result = pd.DataFrame(result)
                    # Save to file
                    df_result.to_csv(str(filename))




Check Run 1 Number 1
  Already aggregated, skip
Check Run 1 Number 10
  Already aggregated, skip
Check Run 1 Number 100
  Already aggregated, skip
Check Run 1 Number 101
  Already aggregated, skip
Check Run 1 Number 102
  Already aggregated, skip
Check Run 1 Number 103
  Already aggregated, skip
Check Run 1 Number 104
  Already aggregated, skip
Check Run 1 Number 105
  Already aggregated, skip
Check Run 1 Number 106
  Already aggregated, skip
Check Run 1 Number 107
  Already aggregated, skip
Check Run 1 Number 108
  Already aggregated, skip
Check Run 1 Number 109
  Already aggregated, skip
Check Run 1 Number 11
  Already aggregated, skip
Check Run 1 Number 110
  Already aggregated, skip
Check Run 1 Number 111
  Already aggregated, skip
Check Run 1 Number 12
  Already aggregated, skip
Check Run 1 Number 13
  Already aggregated, skip
Check Run 1 Number 14
  Already aggregated, skip
Check Run 1 Number 15
  Already aggregated, skip
Check Run 1 Number 16
  Already aggregated, skip
Check Run